In [4]:
import sys, io, os
os.environ["LD_LIBRARY_PATH"] = "/home/renan/MATLAB/R2021b/runtime/glnxa64/";

# btd_rnd e btd_nls
sys.path.append("../matlab/myBTD/myBTD/for_redistribution_files_only");
import myBTD

# ll1_rnd e ll1_nls
sys.path.append("../matlab/myBTD2/myBTD2/for_redistribution_files_only");
import myBTD2

import numpy as np
from myPackages import myDehankelization, myHankelization
import seaborn as sns
import matplotlib.pyplot as plt
import matlab
import pandas as pd
from random import randint
from scipy.io import savemat
from scipy.signal import resample
import ecg_plot
import scipy.fftpack as fftpack
from scipy.stats import kurtosis
from scipy.linalg import norm
import datetime
import csv

In [5]:
def resampling_recording(recording):
    OLD_FREQ = 250
    NEW_FREQ = 100
    leads, sampling = recording.shape
    time_rec = sampling/OLD_FREQ
    n_samples = int(time_rec*NEW_FREQ)
    recording = resample(recording, n_samples, axis=1)
    return recording


def hankelization(recording):
    T = np.zeros((50, 51, recording.shape[0]))
    my_hankelization = myHankelization.initialize()
    for i in range(0, recording.shape[0]):
        vec = matlab.double(recording[i].tolist(), size=recording[i].shape)
        T[:, :, i] = my_hankelization.hankelization(vec)
    my_hankelization.terminate()
    T = matlab.double(T.tolist(), size=T.shape)
    return T


def run_btd(params):
    btd = myBTD.initialize()
    result, output = btd.myBTD(params['rank'], 
                               params['MULTIRANK'], 
                               params['MAX_ITER'], 
                               params['TOLERANCE_FUN'], 
                               params['TOLERANCE_X'], 
                               params['DISPLAY'], 
                               params['recording_hankel'], 
                               nargout=2)
    btd.terminate()
    return result, output
    
    
def run_btd2(params):
    btd = myBTD2.initialize()
    result, output = btd.myBTD2(params['rank'], 
                                params['LR'], 
                                params['MAX_ITER'], 
                                params['TOLERANCE_FUN'], 
                                params['TOLERANCE_X'], 
                                params['DISPLAY'], 
                                params['recording_hankel'], 
                                nargout=2)
    btd.terminate()
    return result, output

In [ ]:
MULTIRANK = matlab.double([40 40 1], size=(1, 3))
LR = matlab.double([40], size=(1, 1))
MAX_ITER = matlab.double([1000], size=(1, 1))
TOLERANCE_FUN = matlab.double([1.0E-6], size=(1, 1))
TOLERANCE_X = matlab.double([1.0E-6], size=(1, 1))
DISPLAY = matlab.logical([False], size=(1, 1))

aldebaran_df = pd.read_pickle(f'../workdata/aldebaran/aldebaran.pkl')

LL1_MODE = False

for rank in range(1, 15):
    print(f'RANK: {rank}')
    
    path = f'../workdata/saga/rank_{rank}'
    if not os.path.exists(path):
        os.makedirs(path)  
        
    rank = matlab.double([rank], size=(1, 1))
    
    for index, row in aldebaran_df.iterrows():
        print(f" - {index}/{aldebaran_df['filename'].count()}")
        recording = row['data']
        recording = resampling_recording(recording)
        recording_hankel = hankelization(recording)
        
        
        params = {
            'rank': rank
            'lr': LR
            'multirank': MULTIRANK
            'max_iter': MAX_ITER
            'tol_fun': TOLERANCE_FUN
            'tol_x': TOLERANCE_X
            'display': DISPLAY
            'recording_hankel': recording_hankel
        }
        
        result, output = run_btd2(params) if LL1_MODE else run_btd(params)

        output['index_aldebaran'] = index
        output['rank'] = rank[0][0]
        output['lr'] = LR[0][0]
        output['filename'] = row['filename']
        output['inf'] = row['inf']
        output['sup'] = row['sup']                        
        output['db'] = row['db']
        output['diagnostic'] = row['diagnostic']
        output['datetime'] = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

        with open(f'{path}/{row.name}.npy', 'wb') as file:
            np.save(file, output)
        
        with open('../workdata/saga/results.csv', 'a') as f:
            headers = output.keys()
            writer = csv.DictWriter(f, delimiter=',', lineterminator='\n', fieldnames=headers)
            if not file_exists:
                writer.writeheader()
            writer.writerow(output.iloc[0].values)